In [1]:
"""Generate a comprehensive circuit guide with diagrams for all 12 benchmark circuits."""

import base64
from io import BytesIO
from pathlib import Path

import numpy as np
from qiskit import QuantumCircuit
import matplotlib.pyplot as plt

# =============================================================================
# Circuit Builders (from benchmark notebook)
# =============================================================================

def build_ghz(n_qubits: int) -> QuantumCircuit:
    """GHZ state: |00...0> + |11...1> / sqrt(2)"""
    qc = QuantumCircuit(n_qubits, name=f'GHZ_{n_qubits}q')
    qc.h(0)
    for i in range(1, n_qubits):
        qc.cx(i - 1, i)
    return qc

def build_bell_pairs(n_pairs: int) -> QuantumCircuit:
    """Parallel Bell pairs."""
    n_qubits = 2 * n_pairs
    qc = QuantumCircuit(n_qubits, name=f'Bell_{n_pairs}pairs')
    for i in range(n_pairs):
        qc.h(2 * i)
        qc.cx(2 * i, 2 * i + 1)
    return qc

def build_ising_trotter(n_qubits: int, steps: int = 3, dt: float = 0.5) -> QuantumCircuit:
    """Trotterized transverse-field Ising model."""
    qc = QuantumCircuit(n_qubits, name=f'Ising_{n_qubits}q_t{steps}')
    J, h = 1.0, 0.5
    for q in range(n_qubits):
        qc.h(q)
    for _ in range(steps):
        for q in range(n_qubits - 1):
            qc.cx(q, q + 1)
            qc.rz(2 * J * dt, q + 1)
            qc.cx(q, q + 1)
        for q in range(n_qubits):
            qc.rx(2 * h * dt, q)
    return qc

def build_h2_ansatz(theta: float = 0.5) -> QuantumCircuit:
    """H2 molecule ansatz (4 qubits)."""
    qc = QuantumCircuit(4, name='H2_ansatz')
    qc.x(0); qc.x(1)
    qc.cx(1, 2); qc.ry(theta, 2); qc.cx(1, 2)
    qc.cx(0, 3); qc.ry(theta / 2, 3); qc.cx(0, 3)
    return qc

def build_lih_ansatz(theta: float = 0.5) -> QuantumCircuit:
    """LiH molecule ansatz (6 qubits)."""
    qc = QuantumCircuit(6, name='LiH_ansatz')
    qc.x(0); qc.x(1)
    for i in range(3):
        qc.cx(i, i + 1)
        qc.ry(theta * (i + 1) / 3, i + 1)
        qc.cx(i, i + 1)
    qc.cx(3, 4); qc.ry(theta / 2, 4); qc.cx(3, 4)
    qc.cx(4, 5); qc.ry(theta / 3, 5); qc.cx(4, 5)
    return qc

def build_qaoa_maxcut_ring(n_qubits: int, p: int = 1, gamma: float = 0.5, beta: float = 0.5) -> QuantumCircuit:
    """QAOA for MAX-CUT on ring graph."""
    qc = QuantumCircuit(n_qubits, name=f'QAOA_ring_{n_qubits}q_p{p}')
    for q in range(n_qubits): qc.h(q)
    for _ in range(p):
        for q in range(n_qubits):
            q_next = (q + 1) % n_qubits
            qc.cx(q, q_next); qc.rz(2 * gamma, q_next); qc.cx(q, q_next)
        for q in range(n_qubits): qc.rx(2 * beta, q)
    return qc

def build_ghz_phase_sensing(n_qubits: int, phi: float = 0.1) -> QuantumCircuit:
    """GHZ state with phase encoding."""
    qc = build_ghz(n_qubits)
    qc.name = f'GHZ_phase_{n_qubits}q'
    for q in range(n_qubits): qc.rz(phi, q)
    return qc

print("Circuit builders defined!")

Circuit builders defined!


In [2]:
# Circuit definitions with metadata
CIRCUITS = {
    "S-GHZ-4": {
        "name": "4-Qubit GHZ State",
        "family": "Shadows Core",
        "family_code": "S",
        "qubits": 4,
        "builder": lambda: build_ghz(4),
        "short_desc": "Maximally entangled 4-qubit Greenberger-Horne-Zeilinger state",
        "description": "The GHZ state is a fundamental quantum state exhibiting genuine multipartite entanglement. For 4 qubits: |GHZ4> = (|0000> + |1111>) / sqrt(2). All qubits are in perfect superposition of all-0 or all-1.",
        "physics": "|GHZ4> = (|0000> + |1111>) / sqrt(2)\nStabilizers: XXXX, ZZII, IZZI, IIZZ\nMaximum multipartite entanglement",
        "applications": "Quantum error correction, quantum metrology (Heisenberg limit), quantum communication, Bell tests",
        "importance": "Tests ability to estimate multi-qubit correlations like <XXXX> where all shadow measurements must align."
    },
    "S-GHZ-5": {
        "name": "5-Qubit GHZ State",
        "family": "Shadows Core",
        "family_code": "S",
        "qubits": 5,
        "builder": lambda: build_ghz(5),
        "short_desc": "Maximally entangled 5-qubit GHZ state",
        "description": "Extension of GHZ to 5 qubits: |GHZ5> = (|00000> + |11111>) / sqrt(2). Larger Hilbert space (32 dims) and more stabilizers to verify.",
        "physics": "|GHZ5> = (|00000> + |11111>) / sqrt(2)\nStabilizers: XXXXX, ZZIII, IZZII, IIZZI, IIIZZ",
        "applications": "5-qubit error correction, distributed quantum computing, 5-party secret sharing",
        "importance": "Tests scaling behavior - 5-qubit operators like XXXXX have 243x higher variance than single-qubit."
    },
    "S-BELL-2": {
        "name": "2 Bell Pairs",
        "family": "Shadows Core",
        "family_code": "S",
        "qubits": 4,
        "builder": lambda: build_bell_pairs(2),
        "short_desc": "Two independent Bell pairs (4 qubits total)",
        "description": "Two independent maximally-entangled Bell pairs: |Phi+>_01 x |Phi+>_23. Bipartite entanglement - qubits entangled in pairs, not globally.",
        "physics": "|Psi> = |Phi+>_01 x |Phi+>_23\nWithin pairs: <ZZ>_01 = <ZZ>_23 = +1\nCross-pairs: <ZZ>_02 = <ZZ>_13 = 0",
        "applications": "Quantum teleportation, superdense coding, entanglement distribution, crosstalk detection",
        "importance": "Tests correct identification of correlation structure - must distinguish within-pair vs cross-pair correlations."
    },
    "S-BELL-3": {
        "name": "3 Bell Pairs",
        "family": "Shadows Core",
        "family_code": "S",
        "qubits": 6,
        "builder": lambda: build_bell_pairs(3),
        "short_desc": "Three independent Bell pairs (6 qubits total)",
        "description": "Three parallel Bell pairs on 6 qubits. Tests shadow estimation on structured 6-qubit state with localized entanglement.",
        "physics": "|Psi> = |Phi+>_01 x |Phi+>_23 x |Phi+>_45\nOnly <ZZ> within each pair is nonzero",
        "applications": "Parallel quantum channels, error characterization, network primitives",
        "importance": "At 6 qubits, shadow variance becomes significant. Tests resolution of pair structure."
    },
    "S-ISING-4": {
        "name": "4-Qubit Ising Ground State",
        "family": "Shadows Core",
        "family_code": "S",
        "qubits": 4,
        "builder": lambda: build_ising_trotter(4, steps=3),
        "short_desc": "Trotterized transverse-field Ising model ground state",
        "description": "Approximate ground state of the transverse-field Ising model (TFIM) via Trotterized evolution. H = -J Sum(ZiZi+1) - h Sum(Xi)",
        "physics": "H = -J Sum(ZiZi+1) - h Sum(Xi), J=1.0, h=0.5\nStrong ZZ correlations: <ZiZi+1> ~ -0.9 to -1.0\nNonzero magnetization: <Xi> ~ 0.3-0.5",
        "applications": "Condensed matter physics, quantum simulation, VQE preparation, quantum annealing",
        "importance": "Ising models have local (nearest-neighbor) correlations. Shadows should efficiently estimate with low variance."
    },
    "S-ISING-6": {
        "name": "6-Qubit Ising Ground State",
        "family": "Shadows Core",
        "family_code": "S",
        "qubits": 6,
        "builder": lambda: build_ising_trotter(6, steps=3),
        "short_desc": "Larger Trotterized Ising ground state approximation",
        "description": "6-qubit TFIM ground state. More qubits means richer correlation structure with 5 nearest-neighbor ZZ pairs.",
        "physics": "6 qubits in a line with 5 ZZ bonds\nq0 -- q1 -- q2 -- q3 -- q4 -- q5\nLonger-range <ZiZj> decay with distance",
        "applications": "Quantum phase transitions, entanglement area law studies, many-body physics",
        "importance": "Tests whether shadows maintain accuracy for longer-range correlations as system size grows."
    },
    "C-H2": {
        "name": "H2 Molecule Ansatz",
        "family": "Chemistry",
        "family_code": "C",
        "qubits": 4,
        "builder": lambda: build_h2_ansatz(0.5),
        "short_desc": "Variational ansatz for hydrogen molecule ground state",
        "description": "Hardware-efficient ansatz for H2 in STO-3G minimal basis. 4 qubits = 4 spin-orbitals: q0,q1 (1s) and q2,q3 (1s* antibonding).",
        "physics": "Molecular Hamiltonian has ~15 Pauli terms\nKey observables: Energy <H>, Particle number <N>, Spin <S^2>",
        "applications": "Variational Quantum Eigensolver (VQE), ground state energy computation, quantum chemistry benchmarks",
        "importance": "Chemistry requires estimating many Pauli terms for energy. Shadows' multi-observable estimation is ideal."
    },
    "C-LiH": {
        "name": "LiH Molecule Ansatz",
        "family": "Chemistry",
        "family_code": "C",
        "qubits": 6,
        "builder": lambda: build_lih_ansatz(0.5),
        "short_desc": "Variational ansatz for lithium hydride ground state",
        "description": "6-qubit ansatz for LiH molecule, more complex than H2. LiH is ionic with partial charge transfer. Hamiltonian has ~100+ Pauli terms.",
        "physics": "Bond length ~1.6 A, Dipole moment ~5.9 Debye\nHamiltonian has O(100) Pauli terms",
        "applications": "Beyond-H2 benchmarks, ionic system studies, active space methods",
        "importance": "With ~100 observables, tests whether shadows provide practical advantage over direct measurement."
    },
    "O-QAOA-5": {
        "name": "QAOA MAX-CUT (5-node Ring)",
        "family": "Optimization",
        "family_code": "O",
        "qubits": 5,
        "builder": lambda: build_qaoa_maxcut_ring(5, p=1),
        "short_desc": "Quantum Approximate Optimization for MAX-CUT on 5-node ring",
        "description": "QAOA for MAX-CUT on 5-node ring graph. Optimal cut is 4 edges (alternating partition). Encodes superposition over all 32 partitions.",
        "physics": "|psi(g,b)> = e^{-ib Sum Xi} e^{-ig Sum ZiZj} |+>^n\nCost: H_C = Sum_{edges} (1 - ZiZj)/2",
        "applications": "Combinatorial optimization, scheduling, logistics, hybrid classical-quantum algorithms",
        "importance": "QAOA observables (ZiZj) all commute, so grouped measurement is efficient. Tests if shadows compete."
    },
    "O-QAOA-7": {
        "name": "QAOA MAX-CUT (7-node Ring)",
        "family": "Optimization",
        "family_code": "O",
        "qubits": 7,
        "builder": lambda: build_qaoa_maxcut_ring(7, p=1),
        "short_desc": "QAOA MAX-CUT on larger 7-node ring",
        "description": "QAOA on 7-node ring. State space is 128-dimensional. Odd nodes cause frustration - best cut is 6 of 7 edges.",
        "physics": "7 nodes in cycle: (0,1),(1,2),...,(6,0)\nOdd ring = frustrated optimization\n128-dim state space",
        "applications": "Scalability testing, frustration effects, parameter landscape studies",
        "importance": "At 7 qubits, benchmark shows shadows having higher variance than direct methods - crossover point."
    },
    "M-PHASE-3": {
        "name": "3-Qubit Phase Sensing",
        "family": "Metrology",
        "family_code": "M",
        "qubits": 3,
        "builder": lambda: build_ghz_phase_sensing(3, phi=0.1),
        "short_desc": "GHZ-enhanced quantum phase estimation",
        "description": "GHZ state with phase rotations for quantum-enhanced sensing. |psi(phi)> = (|000> + e^{3i*phi}|111>) / sqrt(2). Achieves Heisenberg limit.",
        "physics": "|psi(phi)> = (|000> + e^{3i*phi}|111>) / sqrt(2)\n<XXX> = cos(3*phi), <YYY> = sin(3*phi)\nPrecision: delta_phi >= 1/(n*sqrt(M))",
        "applications": "Magnetometry, gravitational wave detection, atomic clocks, quantum radar",
        "importance": "Metrology needs precise estimation of specific observables. Shadows must provide unbiased estimates."
    },
    "M-PHASE-4": {
        "name": "4-Qubit Phase Sensing",
        "family": "Metrology",
        "family_code": "M",
        "qubits": 4,
        "builder": lambda: build_ghz_phase_sensing(4, phi=0.1),
        "short_desc": "4-qubit GHZ-enhanced phase estimation",
        "description": "4-qubit phase sensing. Phase factor e^{4i*phi} provides 4x better resolution than single qubit at same shot count.",
        "physics": "|psi(phi)> = (|0000> + e^{4i*phi}|1111>) / sqrt(2)\n<XXXX> = <YYYY> = cos(4*phi)",
        "applications": "Precision measurement, phase transition detection, quantum sensor networks",
        "importance": "Tests 4-local observable estimation where variance scaling (3^4=81) is significant but manageable."
    }
}

print(f"Defined {len(CIRCUITS)} circuits")

Defined 12 circuits


In [3]:
def circuit_to_base64_image(qc: QuantumCircuit) -> str:
    """Convert a quantum circuit to a base64-encoded PNG image."""
    try:
        fig = qc.draw(output="mpl", style={"backgroundcolor": "#FFFFFF"})
        buf = BytesIO()
        fig.savefig(buf, format="png", dpi=120, bbox_inches="tight", facecolor="white")
        buf.seek(0)
        plt.close(fig)
        return base64.b64encode(buf.read()).decode("utf-8")
    except Exception as e:
        print(f"Error generating image: {e}")
        return None

# Generate all circuit images
circuit_images = {}
for circuit_id, info in CIRCUITS.items():
    qc = info["builder"]()
    img_data = circuit_to_base64_image(qc)
    circuit_images[circuit_id] = img_data
    print(f"Generated diagram for {circuit_id}")

print("\nAll circuit diagrams generated!")

Error generating image: "The 'pylatexenc' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install pylatexenc'."
Generated diagram for S-GHZ-4
Error generating image: "The 'pylatexenc' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install pylatexenc'."
Generated diagram for S-GHZ-5
Error generating image: "The 'pylatexenc' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install pylatexenc'."
Generated diagram for S-BELL-2
Error generating image: "The 'pylatexenc' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install pylatexenc'."
Generated diagram for S-BELL-3
Error generating image: "The 'pylatexenc' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install pylatexenc'."
Generated diagram for S-ISING-4
Error generating image: "The 'pylatexenc' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install pylatexenc'."
Genera

In [4]:
# Generate HTML report
html_parts = ['''<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>QuartumSE Benchmark Circuits - Complete Guide</title>
    <style>
        @page { size: A4; margin: 2cm; }
        body {
            font-family: 'Segoe UI', Arial, sans-serif;
            max-width: 900px;
            margin: 0 auto;
            padding: 30px;
            line-height: 1.7;
            color: #2c3e50;
        }
        h1 {
            color: #2c3e50;
            border-bottom: 3px solid #3498db;
            padding-bottom: 15px;
            font-size: 2.2em;
        }
        h2 {
            color: #34495e;
            margin-top: 50px;
            border-bottom: 2px solid #bdc3c7;
            padding-bottom: 8px;
            font-size: 1.6em;
        }
        .circuit-card {
            background: #f8f9fa;
            border: 1px solid #e9ecef;
            border-radius: 10px;
            padding: 25px;
            margin: 20px 0;
            page-break-inside: avoid;
        }
        .circuit-header {
            display: flex;
            align-items: center;
            margin-bottom: 15px;
            flex-wrap: wrap;
            gap: 10px;
        }
        .family-badge {
            display: inline-block;
            padding: 5px 12px;
            border-radius: 5px;
            font-weight: bold;
            font-size: 0.9em;
            color: white;
        }
        .family-S { background-color: #3498db; }
        .family-C { background-color: #27ae60; }
        .family-O { background-color: #e67e22; }
        .family-M { background-color: #9b59b6; }
        .circuit-title {
            font-size: 1.4em;
            font-weight: bold;
            color: #2c3e50;
        }
        .qubit-count {
            background: #ecf0f1;
            padding: 3px 10px;
            border-radius: 15px;
            font-size: 0.85em;
            color: #7f8c8d;
        }
        .circuit-diagram {
            background: white;
            border: 1px solid #ddd;
            border-radius: 8px;
            padding: 15px;
            margin: 20px 0;
            text-align: center;
        }
        .circuit-diagram img {
            max-width: 100%;
            height: auto;
        }
        .section { margin: 15px 0; }
        .section-title {
            font-weight: bold;
            color: #3498db;
            margin-bottom: 5px;
        }
        .physics-box {
            background: #eef6f9;
            border-left: 4px solid #3498db;
            padding: 15px;
            margin: 15px 0;
            font-family: 'Consolas', monospace;
            font-size: 0.9em;
            white-space: pre-wrap;
        }
        .importance-box {
            background: #fef9e7;
            border-left: 4px solid #f1c40f;
            padding: 15px;
            margin: 15px 0;
        }
        .intro-box {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 15px;
            margin: 30px 0;
        }
        .intro-box h3 { color: white; margin-top: 0; }
        table {
            width: 100%;
            border-collapse: collapse;
            margin: 15px 0;
        }
        th, td { border: 1px solid #ddd; padding: 10px; text-align: left; }
        th { background: #3498db; color: white; }
        tr:nth-child(even) { background: #f9f9f9; }
        .footer {
            margin-top: 50px;
            padding-top: 20px;
            border-top: 1px solid #ddd;
            color: #7f8c8d;
            font-size: 0.9em;
            text-align: center;
        }
    </style>
</head>
<body>

<h1>QuartumSE Benchmark Circuits</h1>
<p><strong>A Complete Guide to the 12 Quantum Circuits</strong></p>
<p style="color: #7f8c8d;">Generated: January 31, 2026 | QuartumSE Classical Shadows Framework</p>

<div class="intro-box">
    <h3>What This Document Covers</h3>
    <p>This guide explains the 12 quantum circuits used in the QuartumSE classical shadows benchmark suite. For each circuit, you will learn:</p>
    <ul>
        <li><strong>What it is:</strong> The quantum state being prepared</li>
        <li><strong>How it works:</strong> The physics and mathematics behind it</li>
        <li><strong>Why it matters:</strong> Real-world applications and importance</li>
        <li><strong>Circuit diagram:</strong> Visual representation of the quantum gates</li>
    </ul>
</div>

<h2>Overview: Circuit Families</h2>
<table>
    <tr><th>Family</th><th>Code</th><th>Circuits</th><th>Purpose</th></tr>
    <tr><td><strong>Shadows Core</strong></td><td>S</td><td>6</td><td>Validate shadow estimation on fundamental entangled states</td></tr>
    <tr><td><strong>Chemistry</strong></td><td>C</td><td>2</td><td>Molecular ground state energy estimation</td></tr>
    <tr><td><strong>Optimization</strong></td><td>O</td><td>2</td><td>QAOA cost function estimation for combinatorial problems</td></tr>
    <tr><td><strong>Metrology</strong></td><td>M</td><td>2</td><td>Quantum-enhanced parameter estimation</td></tr>
</table>
''']

# Add each circuit by family
family_order = [("Shadows Core", "S"), ("Chemistry", "C"), ("Optimization", "O"), ("Metrology", "M")]

for family_name, family_code in family_order:
    family_circuits = [(cid, info) for cid, info in CIRCUITS.items() if info["family_code"] == family_code]
    if not family_circuits:
        continue
    
    html_parts.append(f'''
<h2><span class="family-badge family-{family_code}">{family_code}</span> {family_name} Circuits</h2>
''')
    
    for circuit_id, info in family_circuits:
        img_data = circuit_images.get(circuit_id)
        img_html = f'<img src="data:image/png;base64,{img_data}" alt="{circuit_id} circuit">' if img_data else '<p><em>Diagram generation failed</em></p>'
        
        html_parts.append(f'''
<div class="circuit-card" id="{circuit_id}">
    <div class="circuit-header">
        <span class="family-badge family-{info["family_code"]}">{circuit_id}</span>
        <span class="circuit-title">{info["name"]}</span>
        <span class="qubit-count">{info["qubits"]} qubits</span>
    </div>
    <p><em>{info["short_desc"]}</em></p>
    <div class="circuit-diagram">{img_html}</div>
    <div class="section">
        <div class="section-title">Description</div>
        <p>{info["description"]}</p>
    </div>
    <div class="physics-box">{info["physics"]}</div>
    <div class="section">
        <div class="section-title">Applications</div>
        <p>{info["applications"]}</p>
    </div>
    <div class="importance-box">
        <div class="section-title">Why This Circuit Matters for Benchmarking</div>
        <p>{info["importance"]}</p>
    </div>
</div>
''')

html_parts.append('''
<div class="footer">
    <p><strong>QuartumSE Classical Shadows Benchmark Suite</strong><br>
    This document accompanies the benchmark results from the overnight run of January 30-31, 2026.<br>
    <br>
    For questions or contributions, see the QuartumSE GitHub repository.</p>
</div>
</body>
</html>
''')

# Write HTML file
output_path = Path("circuit_guide_complete.html")
with open(output_path, "w", encoding="utf-8") as f:
    f.write("\n".join(html_parts))

print(f"\nReport generated: {output_path.absolute()}")
print("\nOpen this file in Chrome and use Print > Save as PDF to create the PDF version.")


Report generated: C:\Users\User\Dropbox\QuartumSE\quartumse-internal\notebooks\circuit_guide_complete.html

Open this file in Chrome and use Print > Save as PDF to create the PDF version.
